#### 07 - Serving + Demo (Portfolio Style)

This notebook packages the RAG pipeline into a single callable `ask()` function:
1) Embed the question (Azure OpenAI)
2) Retrieve top-k chunks (Option A brute-force or Option B Vector Search)
3) Build a grounded prompt
4) Generate an answer (Azure OpenAI)
5) Log the request/response to `rag_query_logs`
6) (Optional) Evaluate the answer and write to `rag_evaluations`

Output is a JSON-like dict that can later be used for an API or a simple UI.

In [0]:
%run ./00_install_deps_and_restart

In [0]:
%run ./00_constants

In [0]:
%run ./00_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/948.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 942.1/948.6 kB 60.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/113.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/361.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [0]:
%run ./00_init_openai_client

In [0]:
import mlflow
# Disable mlflow autologging
mlflow.autolog(disable=True)
mlflow.openai.autolog(disable=True)

In [0]:

# Call to ask:

result = ask("What is difference between a normal Azure VM and ephemeral VM?", k=6, retriever="A", do_eval=True, verbose=False)
print("Query ID:", result["query_id"])
print("\nAnswer:\n", result["answer"])

print("\nTop sources:")
for s in result["sources"][:3]:
    print("-", s["url"])

Query ID: 22075cfe-51fb-4d05-98d0-ad1af2142efc

Answer:
 The main differences between a normal Azure VM and an ephemeral VM are related to the storage and persistence of the operating system disk:

1. **Storage Type**:
   - **Normal Azure VM**: A standard Azure VM typically uses a managed OS disk that is persistent. This means that the OS disk retains data even after the VM is stopped or deallocated. The OS disk can be backed up and restored, and it is stored in Azure's managed disk service.
   - **Ephemeral VM**: An ephemeral VM uses an ephemeral OS disk, which is a temporary disk that is not persistent. The data on the ephemeral disk is lost if the VM is stopped or deallocated. Ephemeral disks are designed for scenarios where the OS disk does not need to retain data between reboots, making them suitable for stateless applications.

2. **Performance**:
   - **Normal Azure VM**: The performance of a normal VM can vary based on the type of managed disk used (e.g., Standard HDD, Standard

In [0]:
# Try a few:
qs = [
    "What is an availability set used for?",
    "How do I change VM size in Azure?",
    "How do I create a VM from an image?",
]
for q in qs:
    out = ask(q, k=6, retriever="A", do_eval=True, verbose=False)
    print("\n======================")
    print("Q:", q)
    print("A:", out["answer"][:400], "...")
    print("Faithfulness:", out["eval"]["faithfulness"] if out["eval"] else None)


Q: What is an availability set used for?
A: An availability set is used to provide high availability for applications deployed on Azure virtual machines (VMs). By placing VMs within an availability set, you ensure that during outages or maintenance events that require a reboot, at least one VM remains available. Availability sets achieve this by spreading individual VMs across multiple update domains and fault domains. 

- **Update Domains* ...
Faithfulness: 4

Q: How do I change VM size in Azure?
A: To change the size of a Virtual Machine (VM) in Azure, follow these steps:

### Using the Azure Portal:
1. **Sign in to the Azure Portal**.
2. **Select the VM** from the list of virtual machines.
3. If the VM is running, **stop the VM** by selecting the "Stop" option at the top of the VM overview pane and wait for it to stop.
4. In the VM pane, select **"Size"** from the menu.
5. Choose a new size ...
Faithfulness: 2

Q: How do I create a VM from an image?
A: To create a VM from an image 

In [0]:
# Latest logs

spark.sql(f"""
SELECT
  created_at,
  query_id,
  question,
  top_k,
  retrieved_chunks[0].url AS top_source
FROM {RAG_LOG_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()

created_at,query_id,question,top_k,top_source
2026-01-15T06:21:38.035502Z,aaac6e59-8468-4534-98ec-ea9a10114780,How do I create a VM from an image?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/vm-specialized-image-version.md
2026-01-15T06:21:26.677289Z,5c4969b2-b37f-4a76-8532-31aaf450a73b,How do I change VM size in Azure?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/disks-migrate-lrs-zrs.md
2026-01-15T06:21:14.577388Z,c1f0fdcd-40e3-4de9-a433-50feabc4a7cc,What is an availability set used for?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/virtual-machine-scale-sets-orchestration-modes.md
2026-01-15T06:20:23.192534Z,22075cfe-51fb-4d05-98d0-ad1af2142efc,What is difference between a normal Azure VM and ephemeral VM?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-15T06:20:03.287935Z,0ddcd147-adc0-482a-aab2-94cee6a605a3,What is difference between a normal Azure VM and ephemeral VM?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-15T06:19:13.904144Z,f845afde-9b3d-4719-9ca5-16d984d3bd7e,What is difference between a normal Azure VM and ephemeral VM?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-15T06:18:33.039669Z,721be6a8-494c-45e9-a08f-cf22a960fe8d,What is difference between a normal Azure VM and ephemeral VM?,6,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-15T06:12:18.481359Z,827800ec-0f48-42e2-8ba9-77686e23eba4,What is the difference between Spot VM and normal VM?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/spot-vms.md
2026-01-15T06:12:08.681494Z,603f8671-d355-4e73-a3bf-b60971f83562,How does Azure handle VM disk persistence?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machines/managed-disks-overview.md
2026-01-15T06:12:01.475803Z,bae458ca-5555-4cf2-b40b-a357c3670e78,What is Azure VM Scale Sets?,5,https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/virtual-machine-scale-sets/flexible-virtual-machine-scale-sets-powershell.md


In [0]:
# Latest evaluations

spark.sql(f"""
SELECT
  created_at,
  query_id,
  retrieval_relevance,
  answer_relevance,
  faithfulness,
  notes
FROM {RAG_EVAL_TABLE}
ORDER BY created_at DESC
LIMIT 20
""").display()

created_at,query_id,retrieval_relevance,answer_relevance,faithfulness,notes
2026-01-15T06:21:40.56894Z,aaac6e59-8468-4534-98ec-ea9a10114780,5,5,5,"The retrieved excerpts provide detailed and relevant information on creating a VM from an image using both Azure CLI and Azure PowerShell, which directly supports the answer provided."
2026-01-15T06:21:29.338421Z,5c4969b2-b37f-4a76-8532-31aaf450a73b,2,5,2,"The answer provides a detailed and accurate method for changing the VM size in Azure, which is relevant to the question. However, the retrieved excerpts do not directly support the answer, as they focus on different aspects of VM management, such as disk types and VM creation, rather than resizing VMs."
2026-01-15T06:21:17.477643Z,c1f0fdcd-40e3-4de9-a433-50feabc4a7cc,3,5,4,"The answer is relevant and directly addresses the question about availability sets. However, the retrieved excerpts do not provide specific information about availability sets, which affects the retrieval relevance score. The answer is mostly supported by the context of the excerpts, but there is a slight risk of hallucination due to the lack of direct references."
2026-01-15T06:20:26.330181Z,22075cfe-51fb-4d05-98d0-ad1af2142efc,2,5,2,"The answer provides a detailed comparison between normal Azure VMs and ephemeral VMs, addressing the question well. However, the retrieved excerpts do not contain relevant information about the differences between these two types of VMs, leading to a lower score for retrieval relevance and faithfulness."
2026-01-15T06:20:07.085668Z,0ddcd147-adc0-482a-aab2-94cee6a605a3,2,5,2,"The answer provides a clear and detailed explanation of the differences between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information specifically about ephemeral VMs, leading to a lower score for retrieval relevance and faithfulness."
2026-01-15T06:19:16.867365Z,f845afde-9b3d-4719-9ca5-16d984d3bd7e,2,5,2,"The answer provides a clear and detailed explanation of the differences between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information about ephemeral VMs, focusing instead on managed disks and temporary disks, which affects the faithfulness score."
2026-01-15T06:18:36.874643Z,721be6a8-494c-45e9-a08f-cf22a960fe8d,2,5,2,"The answer provides a clear and detailed explanation of the differences between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information about ephemeral VMs or their differences from normal VMs, leading to a lower score for retrieval relevance and faithfulness."
2026-01-15T06:13:19.304822Z,aa9f09f7-ffca-47b8-a349-98b7cc2cddd8,2,5,3,"The answer provides a clear and detailed explanation of the differences between normal Azure VMs and ephemeral VMs, addressing the question directly. However, the retrieved excerpts do not contain relevant information specifically about ephemeral VMs, which affects the faithfulness score as the answer is not directly supported by the provided excerpts."
2026-01-15T06:13:16.267989Z,5fecd526-9c6b-4afd-b5b6-d381c44f149c,2,5,2,"The answer provides a detailed and accurate process for resizing an Azure virtual machine, which is relevant to the question. However, the retrieved excerpts do not directly support the specific steps mentioned in the answer, particularly regarding the resizing process, leading to a lower faithfulness score."
2026-01-15T06:13:13.362384Z,bae458ca-5555-4cf2-b40b-a357c3670e78,5,5,5,"The retrieved excerpts provide relevant information about Azure VM Scale Sets, including their features and management, which directly supports the answer provided."
